![alt text](<8.png>)
![alt text](<10.png>)
![alt text](<9.png>)

In [1]:
import os 
import openai 

from dotenv import load_dotenv
load_dotenv()

openai.api_key = os.environ['OPENAI_API_KEY']


sk-DGBt3wYwoC-_39R3vkHlXCCzwb3_dSkqb007ngCE3XT3BlbkFJoP06Nzwd0MByr9OSoGdGLzwbXdN2y67TUsqgvNRegA


### Addressing Diversity problem 
### Problem: how to enforce diversity in the search results.

**Maximum marginal relevance strives to achieve both relevance to the query and diversity among the results.**

![alt text](<9.png>)

In [11]:
from langchain.vectorstores import Chroma
from langchain.embeddings.openai import OpenAIEmbeddings
persist_directory = 'files/chroma/'

In [12]:
embedding = OpenAIEmbeddings()
vectordb = Chroma(
    persist_directory=persist_directory,
    embedding_function=embedding
)

In [13]:
print(vectordb._collection.count())

208


In [14]:
texts = [
    """The Amanita phalloides has a large and imposing epigeous (aboveground) fruiting body (basidiocarp).""",
    """A mushroom with a large fruiting body is the Amanita phalloides. Some varieties are all-white.""",
    """A. phalloides, a.k.a Death Cap, is one of the most poisonous of all known mushrooms.""",
]

In [15]:
# loading the text about inforamation to a smal db
smalldb = Chroma.from_texts(texts, embedding=embedding)

In [16]:
question = "Tell me about all-white mushrooms with large fruiting bodies"

In [17]:
smalldb.similarity_search(question, k=2)

[Document(metadata={}, page_content='A mushroom with a large fruiting body is the Amanita phalloides. Some varieties are all-white.'),
 Document(metadata={}, page_content='The Amanita phalloides has a large and imposing epigeous (aboveground) fruiting body (basidiocarp).')]

In [18]:
smalldb.max_marginal_relevance_search(question,k=2, fetch_k=3)

### Conclusion: mmr search diversity 
### Addressing Diversity: Maximum marginal relevance
### Last class we introduced one problem: how to enforce diversity in the search results.

### Maximum marginal relevance strives to achieve both relevance to the query and diversity among the results.

[Document(metadata={}, page_content='A mushroom with a large fruiting body is the Amanita phalloides. Some varieties are all-white.'),
 Document(metadata={}, page_content='A. phalloides, a.k.a Death Cap, is one of the most poisonous of all known mushrooms.')]

In [19]:
question = "what did they say about matlab?"
docs_ss = vectordb.similarity_search(question,k=3)

In [20]:
docs_ss[0].page_content[:100]

'those homeworks will be done in either MATLAB or in Octave, which is sort of — I \nknow some people c'

In [21]:
docs_ss[1].page_content[:100]

'those homeworks will be done in either MATLAB or in Octave, which is sort of — I \nknow some people c'

In [22]:
docs_mmr = vectordb.max_marginal_relevance_search(question,k=3)

In [23]:
#Note the difference in results with MMR. now we have a more diverse set of results
docs_mmr[0].page_content[:100]

'those homeworks will be done in either MATLAB or in Octave, which is sort of — I \nknow some people c'

In [24]:
#Note the difference in results with MMR. now we have a more diverse set of results
docs_mmr[1].page_content[:100]

'into his office and he said, "Oh, professor, professor, thank you so much for your \nmachine learning'

### ADDRESSING THE PROBLEM Addressing Specificity: working with metadata
  We Seen that a question about the third lecture can include results from other lectures as well.

To address this, many vectorstores support operations on metadata.

metadata provides context for each embedded chunk.

In [25]:
question = "what did they say about regression in the third lecture?"

In [29]:
docs = vectordb.similarity_search(
    question,
    k=3,
    filter={"source":"https://see.stanford.edu/materials/aimlcs229/transcripts/MachineLearning-Lecture03.pdf"}
)
# used metadata filter to get the results from the third lecture

In [30]:
for d in docs:
    print(d.metadata)

{'author': '', 'creationdate': '2008-07-11T11:25:03-07:00', 'creator': 'PScript5.dll Version 5.2.2', 'moddate': '2008-07-11T11:25:03-07:00', 'page': 0, 'page_label': '1', 'producer': 'Acrobat Distiller 8.1.0 (Windows)', 'source': 'https://see.stanford.edu/materials/aimlcs229/transcripts/MachineLearning-Lecture03.pdf', 'title': '', 'total_pages': 16}
{'author': '', 'creationdate': '2008-07-11T11:25:03-07:00', 'creator': 'PScript5.dll Version 5.2.2', 'moddate': '2008-07-11T11:25:03-07:00', 'page': 13, 'page_label': '14', 'producer': 'Acrobat Distiller 8.1.0 (Windows)', 'source': 'https://see.stanford.edu/materials/aimlcs229/transcripts/MachineLearning-Lecture03.pdf', 'title': '', 'total_pages': 16}
{'author': '', 'creationdate': '2008-07-11T11:25:03-07:00', 'creator': 'PScript5.dll Version 5.2.2', 'moddate': '2008-07-11T11:25:03-07:00', 'page': 6, 'page_label': '7', 'producer': 'Acrobat Distiller 8.1.0 (Windows)', 'source': 'https://see.stanford.edu/materials/aimlcs229/transcripts/Machin

### Addressing Specificity: working with metadata using self-query retriever
But we have an interesting challenge: we often want to infer the metadata from the query itself.

To address this, we can use SelfQueryRetriever, which uses an LLM to extract:

The query string to use for vector search
A metadata filter to pass in as well
Most vector databases support metadata filters, so this doesn't require any new databases or indexes.

![alt text](<11.png>)

### Additional tricks: compression
Another approach for improving the quality of retrieved docs is compression.

Information most relevant to a query may be buried in a document with a lot of irrelevant text.

Passing that full document through your application can lead to more expensive LLM calls and poorer responses.

Contextual compression is meant to fix this.